In [1]:
import os
import torch
import torch_scatter
import torch_sparse
import torchvision
import torch.nn as nn
import torch_geometric as pyg
from torch.utils.data import Dataset, DataLoader
import torchdata.datapipes as dp
import pandas as pd
import requests
import stat
from tqdm import tqdm
from pathlib import Path
import numpy as np


print(f"PyTorch has version {torch.__version__} with cuda {torch.version.cuda}")


ModuleNotFoundError: No module named 'torch_scatter'

In [11]:
## Import dataset

BASE_URL="https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/${DATASET_NAME}/"



class LearningToSimulateDataset(Dataset):
    def __init__(self, dataset_name, base_url="https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/"):
        self.dataset_name = dataset_name
        self.base_url = base_url

        # Download the dataset if it does not exist.
        if not os.path.exists(f"{dataset_name}.npz"):
            print("Downloading dataset...")
            response = requests.get(f"{base_url}{dataset_name}/train_v1.npz")
            open(f"{dataset_name}.npz", "wb").write(response.content)
            print("Dataset downloaded.")

        # Load the dataset.
        with open(f"{dataset_name}.npz", "rb") as f:
            data = np.load(f)["data"]

        # Convert the data to PyTorch tensors.
        self.inputs = torch.from_numpy(data["input"])
        self.targets = torch.from_numpy(data["target"])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.targets[index]


In [9]:
url = "https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterDropSample/metadata.json"
# response = requests.get(url)
# print(response.ok)
# print(response.status_code)


DATASET_NAME = 'WaterDrop'


BASE_URL = f"https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/{DATASET_NAME}"

# os.chmod(OUTPUT_DIR, stat.S_IRWXU)
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# print(BASE_URL)




file_types = ["metadata.json", "train.tfrecord", "valid.tfrecord", "test.tfrecord"]
for file_type in tqdm(file_types):
    response = requests.get(f"{BASE_URL}/{file_type}")
    output_file = Path(f'/Users/daniel/Documents/Python Projects/pytorchgeometric_tutorial/data/{DATASET_NAME}/{file_type}')
    if not os.path.exists(output_file.parent):
        os.makedirs(output_file.parent)
    with open(output_file, mode='wb') as file:
        file.write(response.content)
        
        


100%|██████████| 4/4 [03:03<00:00, 45.84s/it] 


# Data Preprocessing

In [26]:
def generate_noise(position_seq, noise_std):
    velocity_seq = position_seq[:, 1:] - position_seq[:, :-1]
    time_steps = velocity_seq.size(1)
    velocity_noise = torch.randn_like(velocity_seq) * (noise_std/time_steps ** 0.5)
    

[1 2 3]


In [ ]:

class InteractionNetwork(pyg.nn.MessagePassing):
    def __init__(self, hidden_size, layers=3):
        super().__init__()
        self.lin_edge = torchvision.ops.MLP(hidden_size*3, hidden_size, layers)
        self.lin_node = torchvision.ops.MLP(hidden_size*2, hidden_size,layers)
        return x

    
    def message(self, x_i, x_j, edge_feature):
        x = torch.cat((x_i, x_j, edge_feature), dim=1)
        x = self.lin_edge(x)
        
    def aggregate(self, inputs, index):
        out = torch_sc